In [1]:
import hashlib
import os
import pickle
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import resample
from tqdm import tqdm
from zipfile import ZipFile

In [2]:
def uncompress_files(file):
    features = []
    labels = []
    with ZipFile(file) as zipf:
        filenames_pbar = tqdm(zipf.namelist(), unit='files')
        for filename in filenames_pbar:
            if not filename.endswith('/'):
                with zipf.open(filename) as image_file:
                    image = Image.open(image_file)
                    image.load()
                    feature = np.array(image, dtype=np.float32).flatten()
                label = os.path.split(filename)[1][0]
                features.append(feature)
                labels.append(label)
    return np.array(features), np.array(labels)
train_features, train_labels = uncompress_files('notMNIST_train.zip')
test_features, test_labels = uncompress_files('notMNIST_test.zip')    

train_features, train_labels = resample(train_features, train_labels, n_samples = 150000)
print('All the data is loaded')

100%|███████████████████████████████████████████████████████████████████████| 10001/10001 [00:01<00:00, 5769.70files/s]


All the data is loaded


In [3]:
def normalize_grayscale(image_data):
    a = 0.1
    b = 0.9
    grayscale_min = 0
    grayscale_max = 255
    return a + (((image_data - grayscale_min)*(b - a))/(grayscale_max - grayscale_min))

train_features = normalize_grayscale(train_features)
test_features = normalize_grayscale(test_features)
print('Features are normalized')

Features are normalized


In [4]:
encoder = LabelBinarizer()
encoder.fit(train_labels)
train_labels = encoder.transform(train_labels)
test_labels = encoder.transform(test_labels)

train_labels = train_labels.astype('float32')
test_labels = test_labels.astype('float32')
print('Labels are One-Hot Encoded')

Labels are One-Hot Encoded
